## 1단계

In [1]:
import pandas as pd

titanic_df = pd.read_csv('train.csv')

y_titanic_df = titanic_df['Survived']

x_titanic_df = titanic_df.drop('Survived',axis = 1)

In [2]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
from sklearn.preprocessing import LabelEncoder

def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

def format_features(df):
    df['Cabin']=df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [4]:
x_titanic_df = transform_features(x_titanic_df)
x_titanic_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,7,3
1,1,0,38.0,1,0,71.2833,2,0
2,3,0,26.0,0,0,7.9250,7,3
3,1,0,35.0,1,0,53.1000,2,3
4,3,1,35.0,0,0,8.0500,7,3


## 3 단계

In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_titanic_df,y_titanic_df,test_size=0.2,random_state=11)


## 4단계

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state = 11)
rf_clf = RandomForestClassifier(random_state = 11)
lr_clf = LogisticRegression()

# 결정트리 학습/예측/평가
dt_clf.fit(x_train,y_train)
dt_pred=dt_clf.predict(x_test)
print('결정트리 정확도 : {0:.4f}'.format(accuracy_score(y_test,dt_pred)))

rf_clf.fit(x_train,y_train)
rf_pred=rf_clf.predict(x_test)
print('랜덤포레스트 정확도 : {0:.4f}'.format(accuracy_score(y_test,rf_pred)))

lr_clf.fit(x_train,y_train)
lr_pred=lr_clf.predict(x_test)
print('로지스틱 정확도 : {0:.4f}'.format(accuracy_score(y_test,lr_pred)))

결정트리 정확도 : 0.7877
랜덤포레스트 정확도 : 0.8547
로지스틱 정확도 : 0.8547


C:\Users\SEC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 5단계 교차검증

In [14]:
from sklearn.model_selection import KFold
import numpy as np

def exec_kfold(clf,folds=5):
    kfold = KFold(n_splits=folds)
    scores=[]
    
    for iter_n,(train_index,test_index) in enumerate(kfold.split(x_titanic_df)):
        x_train,x_test = x_titanic_df.values[train_index],x_titanic_df.values[test_index]
        y_train,y_test = y_titanic_df.values[train_index],y_titanic_df.values[test_index]
        
        clf.fit(x_train,y_train)
        pred = clf.predict(x_test)
        accuracy=accuracy_score(y_test,pred)
        scores.append(accuracy)
        print('교차검증 %d : %f' %(iter_n,accuracy))
    
    mean_accuracy = np.mean(scores)
    print('평균 정확도 : %f' % mean_accuracy)

exec_kfold(dt_clf,folds=5)

교차검증 0 : 0.754190
교차검증 1 : 0.780899
교차검증 2 : 0.786517
교차검증 3 : 0.769663
교차검증 4 : 0.820225
평균 정확도 : 0.782299


In [15]:
from sklearn.model_selection import cross_val_score,cross_validate,GridSearchCV

In [16]:
x_titanic_df.values[0]

array([ 3.  ,  1.  , 22.  ,  1.  ,  0.  ,  7.25,  7.  ,  3.  ])

In [18]:
kfold = KFold(n_splits=5)

for train_index,test_index in kfold.split(x_titanic_df):
    print(train_index,test_index)

[179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376
 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394
 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412
 413 414 415 416 417 418 419 420 421 422 423 424 42

In [20]:
scores = cross_val_score(dt_clf,x_titanic_df,y_titanic_df,cv=5)
for iter_n, accuracy in enumerate(scores):
    print('교차검증 {0} 정확도 {1:.4f}'.format(iter_n,accuracy))
print('평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차검증 0 정확도 0.7430
교차검증 1 정확도 0.7753
교차검증 2 정확도 0.7921
교차검증 3 정확도 0.7865
교차검증 4 정확도 0.8427
평균 정확도 : 0.7879


### 6단계 하이퍼파라미터 튜닝

In [25]:
parameters={'max_depth':[2,3,5,10],'min_samples_split':[2,3,5],'min_samples_leaf':[1,5,8]}

grid_dclf=GridSearchCV(dt_clf,param_grid=parameters,scoring='accuracy',cv=5)
grid_dclf.fit(x_train,y_train)

print('GridSearchCV 최적 하이퍼 파라미터 : ',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dclf.best_score_))
best_dclf=grid_dclf.best_estimator_

dpredictions = best_dclf.predict(x_test)
accuracy = accuracy_score(y_test,dpredictions)
print('테스트 세트에서의 DecisionTreeclassifier 정확도 : {0:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.7992
테스트 세트에서의 DecisionTreeclassifier 정확도 : 0.8715


In [26]:
grid_dclf.best_estimator_

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=11)